In [ ]:
println!("Hello Rust")

In [ ]:
:dep supply = { path = "../" }

In [ ]:
:dep chrono = "0.4"
:dep lazy_static = "1.4"
:dep log = "0.4"
:dep env_logger = "0.11.4"
:dep log4rs = "1.2"
:dep serde = { version = "1.0", features = ["derive"] }
:dep serde_yaml = "0.9"
:dep anyhow = "1.0"
:dep tempfile = "3.8"
:dep function_name = "0.3.0"
:dep parking_lot = "0.12"

In [ ]:
use chrono::NaiveDate;
use std::sync::Arc;
use parking_lot::Mutex;

use std::fs;
use function_name::named;

use supply::demand_planner::DemandPlanner;
use supply::demand::Demand;
use supply::sku::SKU;
use supply::specification::Specification;
use supply::flow::Flow;

use supply::simultaneous_flow::SimultaneousFlow;
use supply::operation::{Operation, MaterialFlowVariant, ResourceFlowVariant};
use supply::reports::plan_exporter::PlanExporter;

use supply::resource::Resource;
use supply::resource_flow::ResourceFlow;
use supply::operation::OperationVariant;

use supply::supply_chains::sc_with_alternates::create_sc_with_alternates;

use supply::utilities::traverse::print_supply_chain;
use supply::utilities::cost_calculator::CostCalculator;
use supply::utilities::cost_calculator::PathPrinter;
use supply::utilities::cost_calculator::OperationCost;
use supply::utilities::cost_calculator::SKUCost;
use supply::utilities::cost_calculator::ResourceCost;


use supply::logger_config;
use log::LevelFilter;


In [ ]:
fn create_date(year: i32, month: u32, day: u32) -> NaiveDate {
    NaiveDate::from_ymd_opt(year, month, day).unwrap()
}

In [ ]:
let final_product = create_sc_with_alternates();
let mut cost_calculator = CostCalculator::new();

let _ = logger_config::set_log_level(LevelFilter::Debug);

cost_calculator.set_operation_cost("Make Component A1", OperationCost::new(1.0));
cost_calculator.set_operation_cost("Make Component A2", OperationCost::new(1.0));
cost_calculator.set_operation_cost("Make Component B1", OperationCost::new(1.0));
cost_calculator.set_operation_cost("Make Component B2", OperationCost::new(1.0));
cost_calculator.set_operation_cost("Main Assembly", OperationCost::new(1.0));
cost_calculator.set_operation_cost("Direct Production", OperationCost::new(1.0));
cost_calculator.set_operation_cost("Procure from Main Vendor", OperationCost::new(1.0));
cost_calculator.set_operation_cost("Procure from Alt Vendor", OperationCost::new(1.0));

cost_calculator.set_sku_cost("Raw Material A11", SKUCost::new(1.0));
cost_calculator.set_sku_cost("Raw Material A12", SKUCost::new(1.0));
cost_calculator.set_sku_cost("Raw Material A21", SKUCost::new(1.0));
cost_calculator.set_sku_cost("Raw Material B11", SKUCost::new(1.0));
cost_calculator.set_sku_cost("Raw Material B21", SKUCost::new(1.0));
cost_calculator.set_sku_cost("Raw Material B22", SKUCost::new(1.0));

cost_calculator.set_resource_cost("Assembly_Resource", ResourceCost::new(1.0));

let mut path_printer = PathPrinter::new();
path_printer.collect_alternates(final_product.clone());
let combinations = path_printer.generate_combination_vector();

println!("\nCost Analysis for Each Combination:");
for (i, combo) in combinations.iter().enumerate() {
    println!("\nCombination {} - Operations: {:?}", i + 1, combo);
    path_printer.traverse_selected_path_with_cost(final_product.clone(), combo, &cost_calculator);
}